## Many-to-Many Relationships in SQL

In SQL, many-to-many relationships occur when multiple rows in one table are associated with multiple rows in another table. This relationship is represented using a third table that holds the foreign keys from both tables, creating a link between them.

---

### Example:

Here we have a **Track** table and a **Genre** table:

- **Genre** has a one-to-many relationship with **Track** (one genre can be associated with multiple tracks).
- A single **Track** can belong to many genres, and a **Genre** can have many **Tracks**.

#### Tables involved:

1. **Genre** table: Stores the genre information.
2. **Track** table: Stores track information, including a foreign key pointing to the **Genre** table.

---

## SQL Query for Many-to-Many Relationship:

To retrieve tracks and their genres, we join the **Track** table with the **Genre** table on the `genre_id` column:

```sql
SELECT Track.title, Genre.name
FROM Track
JOIN Genre ON Track.genre_id = Genre.id;
```

## Example: Many-to-Many Relationship with a Junction Table

This schema demonstrates how to model a **many-to-many relationship** in SQL using a **junction table**.

In this example:
- A **User** can be a member of many **Courses**
- A **Course** can have many **Users**
- The **Member** table connects them

---

## User Table

```sql
CREATE TABLE User (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name TEXT,
    email TEXT
);
```

## Explanation

### `id`
- Integer primary key  
- Automatically generated (`AUTOINCREMENT`)  
- Uniquely identifies each user  

### `name`
- User’s name  

### `email`
- User’s email address  

Each user is stored **once** in this table.

---

## Course Table

```sql
CREATE TABLE Course (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title TEXT
);
```
## Explanation

### `id`
- Integer primary key  
- Automatically generated  

### `title`
- Name of the course  

Each course is stored **once** in this table.

---

## Member Table (Junction Table)

```sql
CREATE TABLE Member (
    user_id INTEGER,
    course_id INTEGER,
    role INTEGER,
    PRIMARY KEY (user_id, course_id)
);
```
## Explanation

The **Member** table is the key to the **many-to-many relationship**.

### `user_id`
- References `User.id`
- Identifies which user is involved

### `course_id`
- References `Course.id`
- Identifies which course the user is associated with

### `role`
- Optional attribute describing the user’s role in the course  
  (e.g., student, instructor)

### Composite Primary Key

```sql
PRIMARY KEY (user_id, course_id)
```

- The combination of `(user_id, course_id)` forms the **primary key**
- Ensures that:
  - A user cannot be added to the same course more than once
  - Each user–course relationship is unique
  
## Why This Design Works

### No Duplicated Data
- User details are stored only in the **User** table
- Course details are stored only in the **Course** table

### Flexible Relationships
- A user can join **many courses**
- A course can have **many users**

### Data Integrity
- Relationships are enforced through **primary keys** and **foreign keys**
- Invalid or duplicate relationships are prevented

### Scalable Design
- Additional attributes (such as `role`) can be added to the **Member** table
- The schema can grow without changing existing tables



## Insert Users and Courses

### Inserting Users

The following SQL statements add users to the **User** table:

```sql
INSERT INTO User (name, email) VALUES ('Jane', 'jane@tsugi.org');
INSERT INTO User (name, email) VALUES ('Ed',   'ed@tsugi.org');
INSERT INTO User (name, email) VALUES ('Sue',  'sue@tsugi.org');
```

## Inserting Courses

The following SQL statements add courses to the **Course** table:

```sql
INSERT INTO Course (title) VALUES ('Python');
INSERT INTO Course (title) VALUES ('SQL');
INSERT INTO Course (title) VALUES ('PHP');
```

## Inserting Memberships

After inserting users and courses, we connect them using the **Member** table.  
Each row in this table represents a relationship between a user and a course.

The **Member** table uses:
- `user_id` → references `User.id`
- `course_id` → references `Course.id`
- `role` → indicates the user’s role in the course  
  - `1` = instructor  
  - `0` = student  

### Example Membership Inserts

```sql
-- Course 1: Python
INSERT INTO Member (user_id, course_id, role) VALUES (1, 1, 1);
INSERT INTO Member (user_id, course_id, role) VALUES (2, 1, 0);
INSERT INTO Member (user_id, course_id, role) VALUES (3, 1, 0);

-- Course 2: SQL
INSERT INTO Member (user_id, course_id, role) VALUES (1, 2, 0);
INSERT INTO Member (user_id, course_id, role) VALUES (2, 2, 1);

-- Course 3: PHP
INSERT INTO Member (user_id, course_id, role) VALUES (2, 3, 1);
INSERT INTO Member (user_id, course_id, role) VALUES (3, 3, 0);
```


## Querying a Many-to-Many Relationship with JOIN

Once the **User**, **Course**, and **Member** tables are populated, we can use
`JOIN` to reconstruct meaningful data from these normalized tables.

### The Goal

We want to see:
- User name
- User role in the course
- Course title  

All in a single result set.

---

### SQL Query

```sql
SELECT User.name, Member.role, Course.title
FROM User 
JOIN Member 
JOIN Course
ON Member.user_id = User.id 
AND Member.course_id = Course.id
ORDER BY Course.title, Member.role DESC, User.name;
```

## How This Query Works (Step by Step)

### 1. JOIN User → Member

```sql
Member.user_id = User.id
```

Connects each membership record to the correct user.

Replaces the numeric `user_id` with the user’s name.

### 2. JOIN Member → Course


```sql
Member.course_id = Course.id
```

Connects each membership record to the correct course.

Replaces the numeric `course_id` with the course title.

### 3. Rebuilding the Relationship

The `Member` table acts as the bridge.

It links users and courses together.

This recreates the many-to-many relationship.

### ORDER BY Explained

ORDER BY Course.title, Member.role DESC, User.name

- **Course.title** → group results by course  
- **Member.role DESC** → instructors (`1`) appear before students (`0`)  
- **User.name** → users listed alphabetically within each group

### Example Output

| name | role | title  |
|------|------|--------|
| Jane | 1    | Python |
| Ed   | 0    | Python |
| Sue  | 0    | Python |
| Ed   | 1    | SQL    |
| Sue  | 0    | PHP    |
